# Transform Electrode Coordinates

In [7]:
# %matplotlib inline
%matplotlib qt
%gui qt

# own libraries
import neuropsy as npsy

# standard libraries
import numpy as np

# load T1, make DIG montage, transform coordinates
import nibabel as nib
import mne

# plotting
# import matplotlib.pyplot as plt
# import imageio.v2 as io
# import imageio.v3 as iio
# import os

# 1. Electrode Coordinates

### 1.1 Transform Coordinates

First we load the electrode coordinates for one subject at a time. The coordinates are in scanner RAS space, but to use the coordinates to find labels from the freesurfer parcellation output files - which are in the surface RAS space - we need to transform the elecrode coordinates to the same sruface RAS space.

The MNE-Python toolbox has functions to define a montage which we can attach to the _raw_ object in which we store each subject's data, but these functions expect the electrode coordinates to be represented in meters and furthermore when we apply the montage to the _raw_ object, the coordinates will be automatically transformed to the subject's _head_ space. This is done automatically by using the fsaverage brain's fidicuals to estimate the subject specific fidicuals. And furthermore as we need the coordinates to be in the same space as the freesurfer output, we need to transform the coordinates to surface RAS space. As the fsaverage brain is already in MNI305 space, we can use the subject's MNI transform matrix which has been computed along with the freesurfer output files using the _recon-all_ command. Applying the inverse of this transformation will transform the coordinates from MNI305 to surface RAS space.

In [ ]:
# *************** DEFINES ***************
# paths
path_data       = 'C:/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/data/preprocessed'
subjects_dir    = 'C:/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/data/subjects'
# path_data       = '/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/data/preprocessed'
# subjects_dir    = '/mnt/c/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/data/subjects'
postfix_load    = 'preprocessed'
subject_id      = '19'
subject         = f'sub{subject_id}'
lut_fname       = f'{subjects_dir}/FreeSurferColorLUT.txt'
# [INFO] should 3D image files be saved (True/False)
save_images     = False
display_brain   = True
path_images     = 'C:/Users/matti/OneDrive/Education/SDC/MasterThesis/master-project/results/preprocessing/channel labelling/coordinate transformation'


# *************** LOAD SUBJECT DATA ***************
# load subject data
data = npsy.DataHandler(path=path_data, subject_id=subject_id, exp_phase=2, fs=512, verbose=False)
data.load(load_saved=True, postfix=postfix_load)
data.create_mne_raw()

# load subject MRI data
T1_fname = ''.join([subjects_dir, '/sub', subject_id, '/mri/T1.mgz'])
T1 = nib.load(T1_fname)


# *************** LOAD ELECTRODE COORDINATES ***************
# electrode coordinates are stored in scanner RAS space
coords_ras = []
for ch in data.df_chan['name']:
    coords_ras.append(data.df_chan.loc[data.df_chan['name'] == ch, ['loc_1', 'loc_2', 'loc_3']].to_numpy()[0])
coords_ras = np.asanyarray(coords_ras)

# create montage (coordinates are still in scanner RAS space)
coords_ras_dict = {key: tuple(values) for key, values in zip(data.df_chan['name'], coords_ras)}
montage = mne.channels.make_dig_montage(ch_pos=coords_ras_dict, coord_frame="ras")


# *************** DEFINE TRANSFORMATIONS ***************
# define transformation matrix from mm to m
mm2m = np.eye(4)
mm2m[:3, :3] /= 1000
mm2m_t = mne.transforms.Transform('ras', 'ras', trans=mm2m)

# get transformation matrix from scanner RAS to voxel space
ras2vox = T1.header.get_ras2vox()
ras2vox[:3, :3] *= 1000  # scale from mm to m
ras2vox_t = mne.transforms.Transform(fro="ras", to="mri_voxel", trans=ras2vox)

# get transformation matrix from voxel space to freesurfer surface RAS (MRI) space
vox2mri = T1.header.get_vox2ras_tkr()
vox2mri[:3] /= 1000  # scale from mm to m
vox2mri_t = mne.transforms.Transform(fro="mri_voxel", to="mri", trans=vox2mri)

# combine transforms to get transformation from scanner RAS to freesurfer surface RAS (MRI) space
ras2mri_t = mne.transforms.combine_transforms(ras2vox_t, vox2mri_t, fro="ras", to="mri")

# estimate head to surface RAS (MRI) transformation (mainly for visualization)
head2mri_t = mne.coreg.estimate_head_mri_t(subject=subject, subjects_dir=subjects_dir)
mri2head_t = mne.transforms.invert_transform(head2mri_t)


# *************** APPLY TRANSFORMATIONS ***************
# first transform coordinates from m to mm
montage.apply_trans(mm2m_t)

# then transform coordinates from scanner RAS to freesurfer surface RAS (MRI) space
montage.apply_trans(ras2mri_t)

# transform from surface RAS (MRI) to head space
montage.apply_trans(mri2head_t)

# apply montage to the raw data
data.raw.set_montage(montage)


# *************** VISUALIZE ***************
if display_brain:
    # Areas to highligt in the brain
    labels = (
        "HP_head",
        "HP_body",
        "HP_tail"
        # 'Right-Hippocampus',
    )

    # electrodes to highlight
    # electrodes = ["A", "B", "C"]
    # picks = [
    #     ii
    #     for ii, ch_name in enumerate(data.raw.ch_names)
    #     if any([elec in ch_name for elec in electrodes])]

    # colors for the electrode contacts
    # [INFO] add the channel names here to plot in a different color
    color_picks     = ["C' 02"]
    # [INFO] set color settings
    color_settings  = {"default": "cyan",   
                    "picks": "gold"}
    

    ch_names = data.df_chan['name'].to_list()
    sensor_colors = np.asanyarray([color_settings['default']] * len(ch_names))
    color_idx = np.where(np.isin(ch_names, color_picks))[0]
    sensor_colors[color_idx] = color_settings['picks']


    # Plot the electrode positions in surface RAS space
    fig = mne.viz.plot_alignment(
        info=data.raw.info,
        trans=head2mri_t,
        subject=subject,
        subjects_dir=subjects_dir,
        surfaces=[],
        # sensor_colors='cyan',
        sensor_colors=sensor_colors,
        coord_frame='mri',
        # bgcolor="white"
    )

    fig.plotter.set_background('white')
    # fig.plotter.enable_anti_aliasing()            # Enable anti-aliasing for smoother edges
    # fig.plotter.window_size = [1600, 1200]        # Increase the window size for higher rendering quality

    # Plot the whole brain in an opaque manner
    brain = mne.viz.Brain(
        subject=subject,
        alpha=0.20,
        cortex="low_contrast",
        subjects_dir=subjects_dir,
        units="m",
        figure=fig,
        size=(1200, 1200),
    )

    # Plot the selected brain regions
    # brain.add_volume_labels(aseg="rh.hippoAmygLabels.HBT.resampled", labels=labels, lut_fname=lut_fname)
    # brain.add_volume_labels(aseg="lh.hippoAmygLabels.HBT.resampled", labels=labels, lut_fname=lut_fname)
    brain.add_volume_labels(aseg="hippoAmygLabels.HBT.combined", labels=labels, lut_fname=lut_fname, alpha=0.3)
    # brain.add_volume_labels(aseg="aparc+aseg", labels=labels, lut_fname=lut_fname)
    if not save_images:
        brain.show_view(azimuth=0, elevation=90, distance=0.3)


    if save_images:
        n_steps = 90 
        azimuth_step = 360 / n_steps

        # for second in range(n_seconds):
        for step in range(n_steps):

            # Rotate the brain view
            brain.show_view(azimuth=step * azimuth_step, elevation=90, distance=0.35)
            
            # Save the current frame
            img_path = f"{path_images}/brain gif/frame_{step}.png"
            brain.save_image(img_path)

        del brain

Creating RawArray with float64 data, n_channels=104, n_times=1018626
    Range : 0 ... 1018625 =      0.000 ...  1989.502 secs
Ready.
Channel types::	seeg: 104
    Smoothing by a factor of 0.9


### 1.2 Save to File

Now that we have the electrode coordinates inoculated in the MNE-Python Raw object along with the iEEG data readings we will save the results. But, saving the coordinates to file will be saved in head space, therefore we also need to save the correct transformation matrices to go from head to surface RAS space - which is referred to as MRI space inside MNE documentation - as we need the coordinates in MRI space to match the freesurfer surface output files.

In [25]:
# save raw
data.raw.save(fname=''.join([path_data, '/sub', subject_id, '_ieeg.fif']), 
              split_naming='bids', 
              overwrite=True)

# save transformations
head2mri_t.save(''.join([subjects_dir, '/', subject, '/mri/transforms/t1-head2mri-trans.fif']), overwrite=True)
mri2head_t.save(''.join([subjects_dir, '/', subject, '/mri/transforms/t1-mri2head-trans.fif']), overwrite=True)
ras2mri_t.save(''.join([subjects_dir, '/', subject, '/mri/transforms/t1-ras2mri-trans.fif']), overwrite=True)
mm2m_t.save(''.join([subjects_dir, '/', subject, '/mri/transforms/t1-mm2m-trans.fif']), overwrite=True)

Reserving possible split file sub19_split-01_ieeg.fif
Overwriting existing file.


Writing C:\Users\matti\OneDrive\Education\SDC\MasterThesis\master-project\data\preprocessed\sub19_ieeg.fif
Closing C:\Users\matti\OneDrive\Education\SDC\MasterThesis\master-project\data\preprocessed\sub19_ieeg.fif
[done]
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
